In [1]:
#처음 사용하는 컴퓨터인 경우, OpenCV와 matlab engine 설정 필요

#OpenCV 설정: pip install opencv-python
#matlab engine 설정: https://qlsenddl-lab.tistory.com/21 


In [1]:
velocity = 22 #Digigait 보행 속도 입력

import cv2
import os
import math
import scipy.io
import time
import pandas as pd
import numpy as np
import matlab.engine
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from sklearn.decomposition import PCA

##### OPEN VIDEO FILE #####
Tk().withdraw()
filename = askopenfilename()
print(filename)
head, tail = os.path.split(filename)
csv_file = [f for f in os.listdir(head) if f.startswith(tail[:-4]) & f.endswith('.csv')][0]

##### OPEN CSV AND READ DATA #####
paws = ['LF','LR','RF','RR']
df = pd.read_csv(head+'/'+csv_file)
df = df.rename(columns = df.iloc[0])
df = df.drop([0], axis = 0)
df = df[paws]
df = df.drop([1], axis = 0)
df.head(10)

def getMaxContour(cnts):
    area, idx = -1, -1
    for j in range(len(cnts)):
        M = cv2.moments(cnts[j])
        if M['m00'] > area:
            area = M['m00']
            idx = j
    return cnts[idx]

def hsvFilter(roi, hsv_value, hsv_range):
    blurred = cv2.GaussianBlur(roi, (3, 3), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, centroid_hsv.astype(int) - hsv_range, centroid_hsv.astype(int) + hsv_range)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations = 1)
    mask = cv2.erode(mask, kernel, iterations = 1)
    return mask

def calc_Angle(mask):
    white_pixel_loc = cv2.findNonZero(mask)
    if (type(white_pixel_loc) == type(None)):
        return 0, 0
    
    X = white_pixel_loc.squeeze()
    X[:,1] = -X[:,1]
    
    pca = PCA(n_components=1)
    pca.fit(X)
    X_pca = pca.transform(X)
    X_new = pca.inverse_transform(X_pca)
    X_new = X_new[X_new[:,0].argsort()]
    
    slope = (X_new[-1,1] - X_new[0,1])/(X_new[-1,0] - X_new[0,0])
    angle_in_radians = math.atan(slope)
    angle_in_degrees = math.degrees(angle_in_radians)

    return angle_in_degrees, X_new

##### GET PAW AREA #####
paw_area = [[],[],[],[]]
paw_angle = [[],[],[],[]]
frames_data = [[],[],[],[]]
r = []
frame_count = 0
data = df.to_numpy(dtype=float)
cap = cv2.VideoCapture(filename)
vidlength = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("Start...")

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if len(r) == 0:
        for j in paws:
            r.append(cv2.selectROI(j, frame))
            cv2.destroyAllWindows()
    if ret:
        frame_draw = frame.copy()
        centroids = data[frame_count]
        frame_count += 1
        target_numbers = int(len(centroids)/3)
        for i in range(target_numbers):
            x, y, likelihood = centroids[0+(i*3):3+(i*3)]
            x, y, w, h = int(x), int(y), int(r[i][2]/2), int(r[i][3]/2)
            
            #get centroid_hsv
            centroid_rgb = np.mean(frame[y:y+3, x:x+3], axis = (0,1))
            centroid_hsv = cv2.cvtColor(np.uint8([[centroid_rgb]]), cv2.COLOR_BGR2HSV).squeeze()

            #filter roi
            belowZero = lambda x, y: 0 if (x - y) < 0 else x - y
            roi = frame[belowZero(y, h):y+h, belowZero(x, w):x+w]
            mask = hsvFilter(roi, centroid_hsv, hsv_range = 30)
            angle, line_data = calc_Angle(mask)
            cnts, hier = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            #get area
            if len(cnts) == 0:
                paw_area[i].append(0)
                paw_angle[i].append(0)
            else:
                max_cnt = getMaxContour(cnts)
                M = cv2.moments(max_cnt)
                area = M['m00']

                paw_area[i].append(area)
                paw_angle[i].append(angle)
            
            #draw
            color = (10*i,30*i,60*i)
            cv2.circle(frame_draw, (x,y), 1, color, -1)
            cv2.rectangle(frame_draw, (belowZero(x, w), belowZero(y, h)), (x+w, y+h), color, 2)
            if (type(line_data) != type(0)):
                line_data[:,0] = line_data[:,0] + belowZero(x, w)
                line_data[:,1] = -line_data[:,1] + belowZero(y, h)
                cv2.line(frame_draw, (int(line_data[0,0]), int(line_data[0,1])), (int(line_data[-1,0]), int(line_data[-1,1])), color, 2)
            
            frames_data[i].append(frame_draw)
        if (frame_count % int(vidlength/10) == 0) or (frame_count == vidlength):
            print("Progress: {}/{}".format(frame_count, vidlength))
    else:
        cap.release()
        print("Cap Released.")

print("Done.")


#############SAVE data ###################
scipy.io.savemat('paw_area.mat',
                {'buffer_LF_Area' : paw_area[0],
                 'buffer_LR_Area' : paw_area[1],
                 'buffer_RF_Area' : paw_area[2],
                 'buffer_RR_Area' : paw_area[3],
                 'velocity':velocity})
scipy.io.savemat('paw_angle.mat', 
                 {'buffer_LF_Angle': paw_angle[0],
                  'buffer_LR_Angle': paw_angle[1],
                  'buffer_RF_Angle': paw_angle[2],
                  'buffer_RR_Angle': paw_angle[3]})


while not os.path.exists('./paw_angle.mat'):
    time.sleep(1)
    
print("Matlab Code Running...")

#%%
#CALL MATLAB CODE
eng = matlab.engine.start_matlab()
eng.addpath('./matlabcode',nargout=0)
eng.PawArea_remaker(nargout=0)

final_peak_data = scipy.io.loadmat('./Final_/final_peak_data.mat')
final_angle_data = []
for i in range(len(paws)):
    temp = []
    for j in final_peak_data['locs_upper'+str(i+1)]:
        temp.append(paw_angle[i][int(j)])
    final_angle_data.append(temp)

parent_dir_path = './Paw_Angle_Frame/'
if not os.path.exists(parent_dir_path):
    os.makedirs(parent_dir_path)

Peak_angle_data = [[],[],[],[]]
for i in range(len(paws)):
    paw_dir_path = parent_dir_path + paws[i]
    if not os.path.exists(paw_dir_path):
        os.makedirs(paw_dir_path)
    cnt_2 = 0
    for j in final_peak_data['locs_upper'+str(i+1)]:
        j = int(j)
        fr = frames_data[i][j]
        cv2.imwrite('./Paw_Angle_Frame/'+paws[i]+'/Frame'+str(j)+'.jpg', fr)
        Peak_angle_data[i].append(final_angle_data[i][cnt_2])
        cnt_2 += 1
        

scipy.io.savemat('Peak_paw_angle.mat', 
                 {'buffer_LF_Peak_Angle': Peak_angle_data[0],
                  'buffer_LR_Peak_Angle': Peak_angle_data[1],
                  'buffer_RF_Peak_Angle': Peak_angle_data[2],
                  'buffer_RR_Peak_Angle': Peak_angle_data[3]})

print("Matlab Code Running 2...")

# CALL MATLAB CODE
eng = matlab.engine.start_matlab()
eng.addpath('./matlabcode',nargout=0)
eng.paw_angle_remaker(nargout=0)
print("Done! Close matlab figures to run Data Clean up")

C:/Users/CSP-lab/Desktop/Digigait video/PD76_1.avi
Start...
Progress: 250/2504
Progress: 500/2504
Progress: 750/2504
Progress: 1000/2504
Progress: 1250/2504
Progress: 1500/2504
Progress: 1750/2504
Progress: 2000/2504
Progress: 2250/2504
Progress: 2500/2504
Progress: 2504/2504
Cap Released.
Done.
Matlab Code Running...
Matlab Code Running 2...
Done! Close matlab figures to run Data Clean up


In [2]:
#Data Cleanup
import os, glob
import shutil
target_dir = './'+tail[:-4]
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

f_lst = glob.glob('./*.mat')
for f in f_lst:
    shutil.move(f,target_dir)

############### Move Video File ##################
if not os.path.exists(target_dir+'/Video'):
    os.makedirs(target_dir+'/Video')
    for f in os.listdir(head):
        if f.startswith(tail[:-4]):
            shutil.move(head+'/'+f, target_dir+'/Video')
            
if not os.path.exists(target_dir+'/Final_'):
    shutil.move('./Final_', target_dir)
if not os.path.exists(target_dir+'/Paw_Angle_Frame'):
    shutil.move('./Paw_Angle_Frame', target_dir)            

print("Done!")

Done!
